## Algorytmy grafowe

### Algorytm najbliższego sąsiada

In [18]:
import numpy as np
    
input_data = [
    #A B C D E F
    [0,4,3,1,2,8],#A
    [4,0,7,3,9,1],#B
    [3,7,0,4,3,2],#C
    [1,3,4,0,1,6],#D
    [2,9,3,1,0,7],#E
    [8,1,2,6,7,0],#F
]

def nearest_neighbour(data):
    not_visited = list(np.arange(len(input_data)))

    current_edge = not_visited.pop(0)
    visited = [current_edge]

    min_dist = 1000

    while len(not_visited):
        for edge in not_visited:
            current_dist = data[edge][current_edge]
            if current_dist < min_dist:
                min_dist = current_dist
                next_edge = edge
     

        not_visited.remove(next_edge)
        visited.append(next_edge)
        current_edge = next_edge

        min_dist = 1000

    return visited


path = nearest_neighbour(input_data)

print("Path:")
print(" -> ".join([str(x) for x in path]))


Path:
0 -> 3 -> 4 -> 2 -> 5 -> 1


### Algorytm Kruskala wyznaczania drzewa spinającego

In [7]:
input_data = [
    #A B C D E F
    [0,4,3,1,2,8],#A
    [4,0,7,3,9,1],#B
    [3,7,0,4,3,2],#C
    [1,3,4,0,1,6],#D
    [2,9,3,1,0,7],#E
    [8,1,2,6,7,0],#F
]

class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = []

    def add_edge(self, u, v, w):
        self.graph.append([u, v, w])

    def find(self, parent, i):
        if parent[i] == i:
            return i
        return self.find(parent, parent[i])

    def union(self, parent, rank, x, y):
        x_root = self.find(parent, x)
        y_root = self.find(parent, y)

        if rank[x_root] < rank[y_root]:
            parent[x_root] = y_root
        elif rank[x_root] > rank[y_root]:
            parent[y_root] = x_root
        else:
            parent[y_root] = x_root
            rank[x_root] += 1

    def kruskal(self):
        result = []
        i, e = 0, 0
        self.graph = sorted(self.graph, key=lambda item: item[2])
        parent = []
        rank = []

        for node in range(self.V):
            parent.append(node)
            rank.append(0)

        while e < self.V - 1:
            u, v, w = self.graph[i]
            i += 1
            x = self.find(parent, u)
            y = self.find(parent, v)

            if x != y:
                e += 1
                result.append([u, v, w])
                self.union(parent, rank, x, y)

        print("Minimalne drzewo spinające:")
        for u, v, weight in result:
            print(f"{u} --> {v}, Waga: {weight}")

g = Graph(len(input_data))
for i in range(len(input_data)):
    for j in range(i + 1, len(input_data)):
        if input_data[i][j] != 0:
            g.add_edge(i, j, input_data[i][j])

g.kruskal()

Minimalne drzewo spinające:
0 --> 3, Waga: 1
1 --> 5, Waga: 1
3 --> 4, Waga: 1
2 --> 5, Waga: 2
0 --> 2, Waga: 3


### Algorytm wyznaczania drzewa spinającego

In [1]:
import sys

input_data = [
    #A B C D E F
    [0,4,3,1,2,8],#A
    [4,0,7,3,9,1],#B
    [3,7,0,4,3,2],#C
    [1,3,4,0,1,6],#D
    [2,9,3,1,0,7],#E
    [8,1,2,6,7,0],#F
]

class Prim:
    def __init__(self, graph):
        self.graph = graph
        self.n_apexes = len(graph)
        self.visited = [False] * self.n_apexes
        self.distances = [sys.maxsize] * self.n_apexes
        self.parents = [None] * self.n_apexes

    def find_min_distance(self):
        min_distance = sys.maxsize
        min_index = None
        for v in range(self.n_apexes):
            if not self.visited[v] and self.distances[v] < min_distance:
                min_distance = self.distances[v]
                min_index = v
        return min_index

    def build_mst(self):
        self.distances[0] = 0
        for _ in range(self.n_apexes):
            min_index = self.find_min_distance()
            self.visited[min_index] = True
            for v in range(self.n_apexes):
                if (
                    not self.visited[v]
                    and self.graph[min_index][v] != 0
                    and self.graph[min_index][v] < self.distances[v]
                ):
                    self.distances[v] = self.graph[min_index][v]
                    self.parents[v] = min_index

    def get_minimum_spanning_tree(self):
        minimum_spanning_tree = []
        for v in range(1, self.n_apexes):
            minimum_spanning_tree.append(
                (self.parents[v], v, self.graph[self.parents[v]][v])
            )
        return minimum_spanning_tree


prim = Prim(input_data)
prim.build_mst()
minimum_spanning_tree = prim.get_minimum_spanning_tree()

print("Minimum spanning tree:")

for row in minimum_spanning_tree:
    print(row)

Minimum spanning tree:
(3, 1, 3)
(5, 2, 2)
(0, 3, 1)
(3, 4, 1)
(1, 5, 1)


### Algorytm Johnsona w wersji wykorzystującej algorytmy Dijkstry oraz Bellmana-Forda

In [22]:
input_data = [
    [None, 2, 4, None, None],
    [None, None, 3, 3, None],
    [None, None, None, -1, 4],
    [None, None, None, None, 2],
    [None, None, None, None, None],
]


class JohnsonAlgorithm:
    def __init__(self, graph):
        self.graph = graph

    def dijkstra(self, start_from):
        dist = [float('inf')] * len(self.graph)
        dist[start_from] = 0
        visited = [False] * len(self.graph)

        for i in range(len(self.graph)):
            minimum = float('inf')
            u = 0

            for v in range(len(dist)):
                if minimum > dist[v] and not visited[v]:
                    minimum = dist[v]
                    u = v

            visited[u] = True

            for v in range(len(self.graph)):
                if self.graph[u][v] is not None:
                    if not visited[v] and dist[v] > dist[u] + self.graph[u][v]:
                        dist[v] = dist[u] + self.graph[u][v]

        for i in range(len(dist)):
            if dist[i] != float('inf') and start_from != i:
                print(f"Najszybciej z {start_from} do {i} to dystans {dist[i]}")

    def _bellman_ford(self):
        edges = []
        for i in range(len(self.graph)):
            for j in range(len(self.graph[i])):
                if self.graph[i][j] is not None:
                    edges.append([i, j, self.graph[i][j]])

        for i in range(len(self.graph)):
            edges.append([len(self.graph), i, 0])

        dist = [float('inf')] * (len(self.graph) + 1)
        dist[len(self.graph)] = 0

        for i in range(len(self.graph)):
            for (u, v, w) in edges:
                if dist[u] != float('inf') and dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w

        return dist[0:len(self.graph)]

    def execute(self):
        bellman_dist = self._bellman_ford()

        for i in range(len(self.graph)):
            for j in range(len(self.graph[i])):
                if self.graph[i][j] is not None:
                    self.graph[i][j] = self.graph[i][j] + bellman_dist[i] - bellman_dist[j]

        for i in range(len(self.graph)):
            self.dijkstra(i)




JohnsonAlgorithm(input_data).execute()

Najszybciej z 0 do 1 to dystans 2
Najszybciej z 0 do 2 to dystans 4
Najszybciej z 0 do 3 to dystans 4
Najszybciej z 0 do 4 to dystans 5
Najszybciej z 1 do 2 to dystans 3
Najszybciej z 1 do 3 to dystans 3
Najszybciej z 1 do 4 to dystans 4
Najszybciej z 2 do 3 to dystans 0
Najszybciej z 2 do 4 to dystans 1
Najszybciej z 3 do 4 to dystans 1


### Algorytm kolorowania wierzchołków SL (Smallest Last)

In [24]:
input_data = [
    [0, 1, 1, 0, 1, 0],
    [1, 0, 1, 1, 0, 1],
    [1, 1, 0, 1, 1, 0],
    [0, 1, 1, 0, 0, 1],
    [1, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 1, 0]
]


class SlColoring:
    def __init__(self, g):
        self.g = g
        self.n_apexes = len(g)
        self.colors = [-1] * self.n_apexes
        self.used_colors = set()

    def sort_apexes(self):
        apexes = list(range(self.n_apexes))
        apexes.sort(key=lambda x: sum(self.g[x]))
        return apexes

    def color_apexes(self):
        apexes = self.sort_apexes()
        for apex in apexes:
            available_colors = set(range(self.n_apexes))
            for neighbor in range(self.n_apexes):
                if (
                    self.g[apex][neighbor] == 1
                    and self.colors[neighbor] != -1
                ):
                    available_colors.discard(self.colors[neighbor])
            color = min(available_colors)
            self.colors[apex] = color
            self.used_colors.add(color)

    def get_colors(self):
        return self.colors, len(self.used_colors)

sl_coloring = SlColoring(input_data)
sl_coloring.color_apexes()
colors, num_colors = sl_coloring.get_colors()

print("Min colors:", num_colors)
for apex, color in enumerate(colors):
    print(f"Apex {apex} -> color {color}")

Min colors: 3
Apex 0 -> color 0
Apex 1 -> color 1
Apex 2 -> color 2
Apex 3 -> color 0
Apex 4 -> color 1
Apex 5 -> color 2


### Algorytm kolorowania wierzchołków LF (Largest First)

In [23]:
input_data = [
    [0, 1, 1, 0, 1, 0],
    [1, 0, 1, 1, 0, 1],
    [1, 1, 0, 1, 1, 0],
    [0, 1, 1, 0, 0, 1],
    [1, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 1, 0]
]

class LfColoring:
    def __init__(self, g):
        self.g = g
        self.n_apexes = len(g)
        self.colors = [-1] * self.n_apexes
        self.used_colors = set()

    def sort_apexes(self):
        return sorted(range(self.n_apexes), 
                      key=lambda v: sum(self.g[v]), reverse=True)

    def color_apexes(self):
        apexes = self.sort_apexes()
        for apex in apexes:
            available_colors = set(range(self.n_apexes))
            for neighbor in range(self.n_apexes):
                if (
                    self.g[apex][neighbor] == 1
                    and self.colors[neighbor] != -1
                ):
                    available_colors.discard(self.colors[neighbor])
            current_color = min(available_colors)
            self.colors[apex] = current_color
            self.used_colors.add(current_color)

    def get_colors(self):
        return self.colors, len(self.used_colors)


lf_coloring = LfColoring(input_data)
lf_coloring.color_apexes()
colors, num_colors = lf_coloring.get_colors()

print("Min colors:", num_colors)
for apex, color in enumerate(colors):
    print(f"Apex {apex} -> color {color}")

Min colors: 3
Apex 0 -> color 2
Apex 1 -> color 0
Apex 2 -> color 1
Apex 3 -> color 2
Apex 4 -> color 0
Apex 5 -> color 1
